In [1]:
from langchain_groq import ChatGroq

In [16]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
groq_api_key = os.getenv('GROQ_API_KEY')


In [17]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key = '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',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)
print(groq_api_key)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.
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


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)



Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[]

In [6]:
type(json_res)

list

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
job = json_res
job

[]

In [10]:
# Initialize links outside the loop
links = []

# Ensure that job is a list and iterate over each dictionary in the job list
if job:  # Check if job is not empty
    for item in job:
        skills = item.get('skills', [])  # Get 'skills' from the current item, default to an empty list if not found
        if skills:  # Proceed only if 'skills' is not empty
            # Perform the query
            link = collection.query(query_texts=skills, n_results=2).get('metadatas', [])
            links.append(link)  # Append each result to links

# Now, links will be accessible and contain all results
print(links)


[]


In [11]:
job

[]

In [12]:
if job:
    print(job[0]['skills'])  # Only proceed if the list is not empty
else:
    print("The job list is empty.")


The job list is empty.


In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Shubham Joshi, a final year student at IIIT Jabalpur. Shubham is an data science & ai and Analyst person dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over my experience, he have empowered numerous projects with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the Hiring manager regarding the job mentioned above describing the capability of Shubham
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Business Efficiency with Data Science & AI Expertise

Dear [Hiring Manager's Name],

I am writing to express my enthusiasm for the [Job Title] role at [Company Name]. As a final year student at IIIT Jabalpur, with a strong background in Data Science & AI, I am confident that my skills and experience make me an ideal fit for this position.

With a proven track record of empowering numerous projects with tailored solutions, I have consistently demonstrated my ability to facilitate the seamless integration of business processes through automated tools. My expertise in data analysis, machine learning, and process optimization has enabled me to drive scalability, cost reduction, and heightened overall efficiency in various projects.

As a dedicated Data Science & AI Analyst, I have developed a unique ability to identify areas of improvement and implement data-driven solutions that drive business growth. My proficiency in tools such as Python, R, Tableau, and Power BI has 